In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.transforms as T
from PIL import Image
from torch import nn
from torchvision import models

In [2]:
class ClsNet(nn.Module):
    def __init__(self, **kwargs):
        super(
            ClsNet,
            self,
        ).__init__()

        # Classifier
        self.classifier = nn.Linear(3 * 384 * 256, 9, bias=False)

    def forward(self, img):  # (batch_size, dim)
        bs = img.size(0)
        img = img.flatten(1)
        cls_score = self.classifier(img)  # ([N, num_classes]）
        return cls_score

In [3]:
img = torch.zeros(10, 3, 384, 256)
model = ClsNet()
out = model(img)
print(out.size())
torch.save(model, "/home/hy/project/reid_task/analysis/tmp/cls.pt")

torch.Size([10, 9])


In [9]:
device = "cpu"  # 'cuda' or 'cpu'
loaded_model = torch.load("/home/hy/project/reid_task/analysis/tmp/cls.pt").to(device)
img_path = "/home/hy/project/reid_task/analysis/tmp/0001_c1s1_001051_00.jpg"
transforms = T.Compose(
    [
        T.Resize((384, 256), interpolation=3),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
# read_im = Image.open(img_path).convert("RGB")  # (128, 64, 3)
read_im = Image.fromarray(np.uint8(np.random.randn(128, 64, 3)))  # (128, 64, 3)
trans_img = transforms(read_im).unsqueeze(0).to(device)  # (1, 3, 384, 256)
cls_score = loaded_model(trans_img)
cls_class = torch.argmax(torch.softmax(cls_score, dim=1)).item()
cls_class

3